<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/final/single_cls_roberta_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers
!pip install -q peft
!pip install -q evaluate
!pip install torch tqdm datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import Dataset, DatasetDict
from datasets import load_dataset, concatenate_datasets
from datasets import ClassLabel, Value
from collections import Counter

sof_dataset_train = load_dataset(
    'csv',
    data_files='/content/drive/MyDrive/LLMEmbed/dataset/stackexchange_train_dataset_10000_title.csv',
    split='train'
)


# 결과를 확인
print(sof_dataset_train)

# 레이블의 데이터 건수 카운트
label_counter = Counter(example['label'] for example in sof_dataset_train)

print(label_counter)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['title_content', 'label', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS'],
    num_rows: 10000
})
Counter({0: 2000, 1: 2000, 2: 2000, 3: 2000, 4: 2000})


In [3]:
from datasets import Dataset, DatasetDict
from datasets import load_dataset, concatenate_datasets
from datasets import ClassLabel, Value
from collections import Counter

sof_dataset_val = load_dataset(
    'csv',
    data_files='/content/drive/MyDrive/LLMEmbed/dataset/stackexchange_val_dataset_1000_title.csv',
    split='train'
)


# 결과를 확인
print(sof_dataset_val)

# 레이블의 데이터 건수 카운트
label_counter = Counter(example['label'] for example in sof_dataset_val)

print(label_counter)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['title_content', 'label', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS'],
    num_rows: 1000
})
Counter({0: 200, 1: 200, 2: 200, 3: 200, 4: 200})


In [7]:
from datasets import Dataset, DatasetDict
from datasets import load_dataset, concatenate_datasets
from datasets import ClassLabel, Value
from collections import Counter

sof_dataset_test = load_dataset(
    'csv',
    data_files='/content/drive/MyDrive/LLMEmbed/dataset/stackexchange_test_dataset_1000_title.csv',
    split='train'
)


# 결과를 확인
print(sof_dataset_test)

# 레이블의 데이터 건수 카운트
label_counter = Counter(example['label'] for example in sof_dataset_test)

print(label_counter)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['title_content', 'label', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS'],
    num_rows: 1000
})
Counter({0: 200, 1: 200, 2: 200, 3: 200, 4: 200})


# Tokenizing

In [9]:
from transformers import AutoTokenizer

sof_dataset= DatasetDict({
    "train": sof_dataset_train,
    "val" : sof_dataset_val,
    "test": sof_dataset_test
})

# DatasetDict 출력
print(sof_dataset)


tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large")

def tokenize_function(examples):
    return tokenizer(examples["title_content"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = sof_dataset.map(tokenize_function, batched=True)


DatasetDict({
    train: Dataset({
        features: ['title_content', 'label', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS'],
        num_rows: 10000
    })
    val: Dataset({
        features: ['title_content', 'label', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['title_content', 'label', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS'],
        num_rows: 1000
    })
})


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

# Model / LoRA config

In [10]:
from peft import LoraConfig, TaskType
from peft import get_peft_model
from transformers import RobertaForSequenceClassification

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=8,lora_alpha=16, lora_dropout=0.1
)

model = RobertaForSequenceClassification.from_pretrained(
    'FacebookAI/roberta-large',
    num_labels=5
)

model = get_peft_model(model, lora_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Fine-tuning

In [11]:
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer

import wandb

# wandb 초기화 (API 키가 환경 변수에 설정되어 있어야 함)
# wandb.init(project="my_project", entity="my_entity", name="my_experiment")
wandb.init(project="huggingface") # Uncomment to use wandb


accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metrics = {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "macro_precision": precision_metric.compute(predictions=predictions, references=labels, average="macro")["precision"],
        "macro_recall": recall_metric.compute(predictions=predictions, references=labels, average="macro")["recall"],
        "macro_f1": f1_metric.compute(predictions=predictions, references=labels, average="macro")["f1"],
        "micro_precision": precision_metric.compute(predictions=predictions, references=labels, average="micro")["precision"],
        "micro_recall": recall_metric.compute(predictions=predictions, references=labels, average="micro")["recall"],
        "micro_f1": f1_metric.compute(predictions=predictions, references=labels, average="micro")["f1"],
        "weighted_precision": precision_metric.compute(predictions=predictions, references=labels, average="weighted")["precision"],
        "weighted_recall": recall_metric.compute(predictions=predictions, references=labels, average="weighted")["recall"],
        "weighted_f1": f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"],
    }
    return metrics

training_args = TrainingArguments(output_dir="/content/drive/MyDrive/LoRA/CLASSIFICATION1_roberta_LoRA_stackexchange_FINAL",
                                  evaluation_strategy="epoch",
                                  num_train_epochs=10,
                                  report_to="wandb",  # wandb를 사용하도록 설정
                                  )


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["val"],
    compute_metrics=compute_metrics,
)

trainer.train()

eval/accuracy,▁▅█▅
eval/loss,█▂▁▅
eval/macro_f1,▁▄█▅
eval/macro_precision,▁▄█▅
eval/macro_recall,▁▅█▅
eval/micro_f1,▁▅█▅
eval/micro_precision,▁▅█▅
eval/micro_recall,▁▅█▅
eval/runtime,█▆▁▃
eval/samples_per_second,▁▃█▆
eval/steps_per_second,▁▃█▆


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Macro Precision,Macro Recall,Macro F1,Micro Precision,Micro Recall,Micro F1,Weighted Precision,Weighted Recall,Weighted F1
1,0.208700,0.157988,0.963000,0.963663,0.963000,0.962937,0.963000,0.963000,0.963000,0.963663,0.963000,0.962937
2,0.172200,0.127975,0.968000,0.968118,0.968000,0.967891,0.968000,0.968000,0.968000,0.968118,0.968000,0.967891
3,0.160000,0.122022,0.980000,0.980148,0.980000,0.979939,0.980000,0.980000,0.980000,0.980148,0.980000,0.979939
4,0.117600,0.144326,0.970000,0.970375,0.970000,0.970038,0.970000,0.970000,0.970000,0.970375,0.970000,0.970038
5,0.110900,0.142556,0.973000,0.973225,0.973000,0.972974,0.973000,0.973000,0.973000,0.973225,0.973000,0.972974
6,0.089900,0.144096,0.975000,0.975252,0.975000,0.974958,0.975000,0.975000,0.975000,0.975252,0.975000,0.974958
7,0.088000,0.142000,0.977000,0.977290,0.977000,0.976941,0.977000,0.977000,0.977000,0.977290,0.977000,0.976941
8,0.081000,0.149799,0.974000,0.974180,0.974000,0.973951,0.974000,0.974000,0.974000,0.974180,0.974000,0.973951
9,0.070500,0.138746,0.979000,0.979074,0.979000,0.978958,0.979000,0.979000,0.979000,0.979074,0.979000,0.978958
10,0.074500,0.142506,0.977000,0.977156,0.977000,0.976920,0.977000,0.977000,0.977000,0.977156,0.977000,0.976920


TrainOutput(global_step=12500, training_loss=0.13388181976318358, metrics={'train_runtime': 5302.04, 'train_samples_per_second': 18.861, 'train_steps_per_second': 2.358, 'total_flos': 2.34399211008e+16, 'train_loss': 0.13388181976318358, 'epoch': 10.0})

In [12]:
# 테스트 세트 평가
test_results = trainer.evaluate(tokenized_datasets["test"])
print(test_results)

# WandB에 테스트 결과 로깅
wandb.log(test_results)
wandb.finish()

{'eval_loss': 0.1709912121295929, 'eval_accuracy': 0.965, 'eval_macro_precision': 0.9651231625577358, 'eval_macro_recall': 0.9650000000000001, 'eval_macro_f1': 0.9649666777705317, 'eval_micro_precision': 0.965, 'eval_micro_recall': 0.965, 'eval_micro_f1': 0.965, 'eval_weighted_precision': 0.9651231625577358, 'eval_weighted_recall': 0.965, 'eval_weighted_f1': 0.9649666777705318, 'eval_runtime': 24.0794, 'eval_samples_per_second': 41.529, 'eval_steps_per_second': 5.191, 'epoch': 10.0}


epoch,▁
eval/accuracy,▁▃█▄▅▆▇▆█▇▂
eval/loss,▆▂▁▄▄▄▄▅▃▄█
eval/macro_f1,▁▃█▄▅▆▇▆█▇▂
eval/macro_precision,▁▃█▄▅▆▇▅█▇▂
eval/macro_recall,▁▃█▄▅▆▇▆█▇▂
eval/micro_f1,▁▃█▄▅▆▇▆█▇▂
eval/micro_precision,▁▃█▄▅▆▇▆█▇▂
eval/micro_recall,▁▃█▄▅▆▇▆█▇▂
eval/runtime,▅██▄▃▃▄▂▂▁▁
eval/samples_per_second,▄▁▁▅▆▆▅▇▇██


# Inference

In [13]:
from transformers import pipeline

classifier = pipeline("text-classification", model=trainer.model, tokenizer=tokenizer, device=0)

label_map = {
    0: "biology",
    1: "cooking",
    2: "diy",
    3: "travel",
    4: "stackoverflow"
}

# 주어진 텍스트를 토대로 예측을 5번 실행
texts = [
    "What are the theoretical and actual (measured) minimum water potentials in plants?",
    "What tastes like marigold? <p>I'm  looking to make Georgian Satsivi ",
    "How can I intentionally make my toilet make this noise? <p>For Halloween I want to haunt my toilets and have them make the noise heard here:",
    "What are the hours of operation of the Ankara Metro? <p>What are the times of the first and last trains on the Ankara Metro?</p>",
    "How to use \"HTML form target self\" ? I am new to HTML and need to complete a simple task.",
]

# 각 텍스트에 대해 예측 실행 및 레이블 출력
for i, text in enumerate(texts):
    prediction = classifier(text)
    predicted_label = prediction[0]['label']
    predicted_score = prediction[0]['score']

    # 레이블 이름을 가져옵니다 (여기서는 RoBERTa 모델의 레이블이 "LABEL_0", "LABEL_1" 등으로 되어있을 수 있으므로 매핑이 필요합니다).
    label_name = label_map[int(predicted_label.split('_')[-1])]
    print(f"Text {i+1}: Predicted label: {label_name}, Score: {predicted_score:.4f}")

The model 'PeftModelForSequenceClassification' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GemmaForSequenceClassification'

Text 1: Predicted label: biology, Score: 1.0000
Text 2: Predicted label: cooking, Score: 0.9990
Text 3: Predicted label: diy, Score: 0.9996
Text 4: Predicted label: travel, Score: 0.9966
Text 5: Predicted label: stackoverflow, Score: 1.0000
